# Next word Predictor with LSTM Neural Network

This project demonstrates how to generate text using an LSTM (Long Short-Term Memory) neural network. 

In [1]:
import random
import pickle
import re

import numpy as np
import pandas as pd
from nltk.tokenize import RegexpTokenizer

from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import LSTM, Dense, Activation
from tensorflow.keras.optimizers import RMSprop

In [2]:
with open('1661-0.txt', 'r', encoding='utf-8') as file:
    text = file.read()

clean_text = re.sub(r'[^a-zA-Z\s]', '', text).lower()[:60000]

In [3]:
tokenizer = RegexpTokenizer(r'\w+')
tokens = tokenizer.tokenize(clean_text)

In [4]:
unique_tokens = np.unique(tokens)
unique_token_index = {token: idx for idx, token in enumerate(unique_tokens)}

In [5]:
n_words = 10
input_words = []
next_words = []

for i in range(len(tokens) - n_words):
    input_words.append(tokens[i:i + n_words])
    next_words.append(tokens[i + n_words])

In [6]:
X = np.zeros((len(input_words), n_words, len(unique_tokens)), dtype=bool)
y = np.zeros((len(next_words), len(unique_tokens)), dtype=bool)

In [7]:
for i, words in enumerate(input_words):
    for j, word in enumerate(words):
        X[i, j, unique_token_index[word]] = 1
    y[i, unique_token_index[next_words[i]]] = 1

In [8]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)


In [9]:
model = Sequential()
model.add(LSTM(128, input_shape=(n_words, len(unique_tokens)), return_sequences=True))
model.add(LSTM(128))
model.add(Dense(len(unique_tokens)))
model.add(Activation("softmax"))
model.compile(loss="categorical_crossentropy", optimizer=RMSprop(learning_rate=0.01), metrics=["accuracy"])


C:\Users\Anurag Sunil\AppData\Roaming\Python\Python311\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [17]:
history = model.fit(X, y, batch_size=128, epochs=30, shuffle=True,validation_data=(X_val, y_val))
model.save("mymodel.keras")
model = load_model("mymodel.keras")


Epoch 1/30
90/90 ━━━━━━━━━━━━━━━━━━━━ 7s 48ms/step - accuracy: 0.1463 - loss: 4.9627 - val_accuracy: 0.1873 - val_loss: 4.3646
Epoch 2/30
90/90 ━━━━━━━━━━━━━━━━━━━━ 4s 42ms/step - accuracy: 0.1811 - loss: 4.4552 - val_accuracy: 0.2178 - val_loss: 4.0546
Epoch 3/30
90/90 ━━━━━━━━━━━━━━━━━━━━ 4s 42ms/step - accuracy: 0.2115 - loss: 4.1814 - val_accuracy: 0.2313 - val_loss: 3.8811
Epoch 4/30
90/90 ━━━━━━━━━━━━━━━━━━━━ 4s 42ms/step - accuracy: 0.2565 - loss: 3.8774 - val_accuracy: 0.2879 - val_loss: 3.5776
Epoch 5/30
90/90 ━━━━━━━━━━━━━━━━━━━━ 4s 42ms/step - accuracy: 0.2858 - loss: 3.6463 - val_accuracy: 0.3166 - val_loss: 3.2957
Epoch 6/30
90/90 ━━━━━━━━━━━━━━━━━━━━ 4s 43ms/step - accuracy: 0.3346 - loss: 3.3694 - val_accuracy: 0.4055 - val_loss: 2.9312
Epoch 7/30
90/90 ━━━━━━━━━━━━━━━━━━━━ 4s 42ms/step - accuracy: 0.3845 - loss: 3.0682 - val_accuracy: 0.4190 - val_loss: 2.7389
Epoch 8/30
90/90 ━━━━━━━━━━━━━━━━━━━━ 4s 42ms/step - accuracy: 0.4278 - loss: 2.8012 - val_accuracy: 0.5048 - v

In [12]:
def predict_next_word(input_text, n_best):
    input_text = input_text.lower()
    X = np.zeros((1, n_words, len(unique_tokens)))
    for i, word in enumerate(input_text.split()):
        X[0, i, unique_token_index[word]] = 1
    predictions = model.predict(X)[0]
    return np.argpartition(predictions, -n_best)[-n_best:]

In [13]:
possible = predict_next_word("He will have to do that thing for it is", 5)
print([unique_tokens[idx] for idx in possible])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 315ms/step
['be', 'me', 'my', 'the', 'a']


## Additional Task 

In [14]:
def generate_text(input_text, text_length, creativity=3):
    word_sequence = input_text.split()
    current = 0
    for _ in range(text_length):
        sub_sequence = " ".join(tokenizer.tokenize(" ".join(word_sequence).lower())[current:current + n_words])
        try:
            choice = unique_tokens[random.choice(predict_next_word(sub_sequence, creativity))]
        except:
            choice = random.choice(unique_tokens)
        word_sequence.append(choice)
        current += 1
    return " ".join(word_sequence)

In [16]:
print(generate_text("He will have to do that thing for it is", 100, 5))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━